In [1]:
''' If you have issues with unfound packages, uncomment the pip install lines below'''
# ! pip install ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension
# ! pip install matplotlib
# ! pip install pandas
''' If you have issues with unfound packages, uncomment the pip install lines above'''

' If you have issues with unfound packages, uncomment the pip install lines above'

In [2]:
from ipywidgets import *
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
from enum import Enum
from ipywidgets import HBox, Label

# %matplotlib inline

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
class FileType(Enum):
    UNSUPPORTED = 0
    FASTA = 1
    GFT = 2
    
# TODO use this object to keep track of state information?
class State:
    fasta_file = None
    gft_file = None
    exon_start = None
    display_objects = {}
    
    def state_complete():
        return fasta_file is None
    
summary = {}

In [5]:
def graph_data(series):
    plt.plot(series)
    plt.ylabel('Y label')
    plt.xlabel('X label')
    plt.title('Title')
    plt.show()
    return plt

def read_gft(filename):
    gft = pd.read_csv(filename, sep=r'[;\t]', 
                     header=None, 
                     names=["seqname", "source", "feature", "coordinate1", "coordinate2", "score", "orientation", "frame", 
                            "gene_id", "transcript_id", "drop_me"],
                     engine='python')
    gft = gft.drop("drop_me", axis=1)
    return gft

def get_avg_gene_span(gft):
    spans = []
    for gid in gft.gene_id.unique():
        exon_coordinate_list = (gft[ gft.gene_id == gid].coordinate1).append(gft[ gft.gene_id == gid].coordinate2)
        gene_span = exon_coordinate_list.max() - exon_coordinate_list.min() + 1
        spans.append(gene_span)
    return pd.Series(spans).mean()

def get_avg_intergenic_space(gft):
    # intergenic space is min cordinate 1 of gene 2      max cordinate 2 of gene 1
    spacing = pd.DataFrame()
    spacing["left"] = gft.groupby("gene_id").coordinate1.min()
    spacing["right"] = gft.groupby("gene_id").coordinate1.max()
    diff = []
    for r in range(0, len(spacing) - 1):
        diff.append(spacing.iloc[r + 1].right - spacing.iloc[r].left)
    return pd.Series(diff).mean()

def get_avg_length(gft, condition):
    gft = gft[condition]
    spacing = pd.DataFrame()
    spacing["left"] = gft.coordinate1
    spacing["right"] = gft.coordinate2
    diff = []
    for r in range(0, len(spacing) - 1):
            diff.append(spacing.iloc[r].left - spacing.iloc[r + 1].right + 1)
    return pd.Series(diff).mean()

def find_200_before_gene(gft):
    print(gft.head())
    

def process_gft(gft):
    global summary
    summary["Number of Genes"] = gft.gene_id.nunique()
    summary["Gene Density"] = gft.gene_id.nunique() / gft.gene_id.count()
    summary["Average Gene Span"] = get_avg_gene_span(gft[gft.feature != "stop_codon"])
    summary["Average Intergenic Spaces"] = get_avg_intergenic_space(gft) 
    
    summary["Average Intron Length"] = get_avg_length(gft, gft.feature == "exon")
    
    summary["Number of Exons"] = gft[gft.feature == "exon"].seqname.count()
    summary["Number of Introns"] = summary["Number of Exons"] - 1
    summary["Average Exon Length"] = (gft[gft.feature != "stop_codon"].coordinate2 - gft[gft.feature != "stop_codon"].coordinate1 + 1).mean()
    
    summary["Average GC Content"] = "Use information from the fasta file!"
    return summary
    
def read_lab1(filename):
    seq = ""
    with open(filename) as file:
        line1 = file.readline()
        seq = file.readline().upper()
    d = {'A': [], 'C': [], 'G': [], 'T': [], 'N': []}
    for s in seq:
        d[s].append(1)
        for k in d.keys():
            if k is not s:
                d[k].append(0)
    df = pd.DataFrame(d)
    return df
    
def analyze_lab1(df):
    total_gc = df.C.mean() + df.G.mean()
    global summary 
    summary["Average GC Content:"] = total_gc
    summary["Total Unknown:"] = df.N.mean()
    
    s = int(df.shape[0])
    window_size = int(s / 20)
    plt.plot(total_gc)
    off = int(s/float(66.6))

    # TODO make this better. 
    if off == 0:
        plt.plot(get_averages(df, s, window_size, offset=0)[0])
    else:
        for o in range(0, window_size, off):
            plt.plot(get_averages(df, s, window_size, offset=o)[0])

    plt.ylabel('Percentage')
    plt.xlabel('Window Number')
    plt.title('GC Percentage')
    plt.show()
    return plt

def get_averages(df, s, window_size, offset=0, print_me=False):
    rolling_avg = []
    window = []
    for i in range(offset, s, window_size):
        gc = df.iloc[i : i + window_size].G.mean() + df.iloc[i : i + window_size].C.mean()
        if print_me:
            print("window: ", i, i + window_size, "\t the gc content is: ", gc, "unknown: ", df.iloc[i : i + window_size].N.mean())
        rolling_avg.append(gc)
        window.append((i, i+window_size))
    return rolling_avg, window
    

def handle_gft(fname):
    filename = "data/" + fname
    file_type = FileType.GFT
    with open(filename) as file:
        line1 = file.readline()
        # TODO some type check for is GFT or GFF file!
        if line1[0] is '>':
            raise UserWarning("That file type is unsupported. Exiting.")
            sys.exit(-1)
    df = read_gft(filename)
    summary = process_gft(df)
    return summary

def handle_fasta(fname):
    filename = "data/" + fname
    file_type = FileType.FASTA
    with open(filename) as file:
        line1 = file.readline()
        # TODO additional type check for is GFT or GFF file!
        if line1[0] is not '>':
            raise UserWarning("That file type is unsupported. Exiting.")
            sys.exit(-1)
    df = read_lab1(filename)
    analyze_lab1(df)
    #     plt_instance = graph_lab1(df)

    print("\n".join("{}: {}".format(k, v) for k, v in summary.items()))
    #     return plt_instance

In [6]:
# This is the display UI code which calls event handler code
# get all the file names in the data folder and display for our users
files = [f for f in os.listdir('data') if os.path.isfile(os.path.join("data", f))]

current = State()
gtf_dropdown = widgets.Dropdown(options= files, description= "GFT:")
fasta_dropdown = widgets.Dropdown(options= files, description= "FASTA:")
exon_start = widgets.BoundedIntText(
    value = 7,
    min = 0,
    max = 10,
    step=1,
    description='Start : ',
    disabled=False
)

current.display_objects["gtf_dropdown"] =  gtf_dropdown
current.display_objects["fasta_dropdown"] = fasta_dropdown
current.display_objects["exon_start"] = exon_start

def is_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        return change['new']
    return None

def fasta_selection(dd):
    if is_change(dd):
        return handle_fasta(is_change(dd))
    return None

def gft_selection(dd):
    if is_change(dd):
        global summary
        summary = handle_gft(is_change(dd))
#         exon_start.max = summary["max"]
#         exon_start.min = summary["min"]

        display(fasta_dropdown)
    return None

o = gtf_dropdown.observe(gft_selection)
o = fasta_dropdown.observe(fasta_selection)

display(gtf_dropdown)

Dropdown(description='GFT:', options=('gft1.txt', 'DNA.txt', 'customgft.txt'), value='gft1.txt')

In [7]:
print("only analyze first 200 nucleotides before each geneid?")


only analyze first 200 nucleotides before each geneid?


In [8]:
seq = ""
with open("data/DNA.txt") as file:
    line1 = file.readline()
    seq = file.readline().upper()
my_sequence = seq

In [9]:
from Bio.Seq import Seq

#create a sequence object
my_seq = Seq('CATGTAGACTAG')

#print out some details about it
print( 'seq %s is %i bases long' % (my_seq, len(my_seq)))
print( 'reverse complement is %s' % my_seq.reverse_complement())
print( 'protein translation is %s' % my_seq.translate())

seq CATGTAGACTAG is 12 bases long
reverse complement is CTAGTCTACATG
protein translation is HVD*


In [149]:
def frequentWords( s, k ):
    counts = {}
    for i in range(0,len(s)-k+1):
        if s[i:i+k] not in counts:
            counts[s[i:i+k]] = 0
        counts[s[i:i+k]] += 1
    m = max(counts.values())
    top_counts = list(counts.values())
    top_counts.sort()
    
    top_counts = top_counts[-3:]
    out = []
    
    print(counts)
    
    for kmer in counts:
        if k > 5 and (counts[kmer] in top_counts) and (counts[kmer] >= 2):
            out.append(kmer)
        elif counts[kmer] in top_counts and (counts[kmer] > 2):
            out.append(kmer)
    return out

In [150]:
def get_motifs_and_locations(f, seq):
    # please dont judge me chris
    motif_location = {}
    k = len(f[0])
    print(f)
    for i in range(0,len(seq) - k + 1):
        window = seq[i : i + k]
        if window in f:
            if window not in motif_location:
                motif_location[window] = []
            motif_location[window].append(i)
    print("\n".join("{}: {}".format(k, v) for k, v in motif_location.items()))

In [148]:
for i in range(3, 11):
    f = frequentWords(my_sequence[:200], i)
    if f:
        get_motifs_and_locations(f, my_sequence[:200])
    else:
        break

['CGG', 'GGA', 'GAA', 'CGA', 'GCG', 'GAG', 'TCA', 'TTT']
CGG: [2, 20, 48, 79, 115, 163]
GGA: [6, 50, 62, 80, 117, 164, 175, 179]
GAA: [7, 26, 71, 157, 180, 194]
CGA: [14, 23, 59, 102, 149, 156]
GCG: [22, 47, 101, 114, 121, 189]
GAG: [24, 51, 60, 63, 99, 118]
TCA: [31, 37, 55, 76, 110, 125]
TTT: [74, 105, 106, 107, 108, 152, 153]
['TGGA', 'GAAT', 'ATGG', 'CGAT', 'TCAC', 'AAGC', 'AGCG', 'GGAG', 'TTCA', 'GAGG', 'AGGC', 'ATTT', 'TTTC', 'TTTT']
TGGA: [5, 174, 178]
GAAT: [7, 26, 71]
ATGG: [9, 93, 177]
CGAT: [14, 102, 149]
TCAC: [37, 55, 76]
AAGC: [45, 85, 112]
AGCG: [46, 100, 113]
GGAG: [50, 62, 117]
TTCA: [54, 75, 109, 124]
GAGG: [60, 63, 118]
AGGC: [64, 119, 137]
ATTT: [73, 104, 151]
TTTC: [74, 108, 153]
TTTT: [105, 106, 107, 152]


In [14]:
gft = pd.read_csv("data/gft1.txt", sep=r'[;\t]', 
                     header=None, 
                     names=["seqname", "source", "feature", "coordinate1", "coordinate2", "score", "orientation", "frame", 
                            "gene_id", "transcript_id", "drop_me"],
                     engine='python')
gft = gft.drop("drop_me", axis=1)

In [15]:
gft.gene_id.unique()

array(['gene_id "Ddx1-RA"', 'gene_id "Rich-RA"', 'gene_id "Rich-RB"'], dtype=object)

In [16]:
starts = {}

mi = gft.groupby("gene_id").min()
ma = gft.groupby("gene_id").max()

for n in ma[ma.orientation == '-'].index:
    starts[str(n)] = ma.loc[n].coordinate2
    # range this to + 200

for n in mi[mi.orientation == '+'].index:
    starts[str(n)] = mi.loc[n].coordinate1
    
    
starts

{'gene_id "Ddx1-RA"': 34124,
 'gene_id "Rich-RA"': 35332,
 'gene_id "Rich-RB"': 35332}